<a href="https://colab.research.google.com/github/lgiesen/forest_height/blob/main/notebooks/PrepareDataForCnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [42]:
import numpy as np
import pandas as pd
import datetime, os, cv2
from matplotlib import pyplot as plt
from matplotlib.ticker import StrMethodFormatter
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error as mse, mean_absolute_error as mae, mean_absolute_percentage_error as mape
from keras.models import Sequential, load_model
from keras.layers import Dense, BatchNormalization, Dropout, InputLayer, Flatten, Conv2D, MaxPool2D
from keras.callbacks import TensorBoard, ModelCheckpoint

In [ ]:
!unzip gdrive/My\ Drive/DataDa2/images_train.zip #first part of the data

In [ ]:
!unzip gdrive/My\ Drive/DataDa2/masks_train.zip

In [5]:
size = 5 #define window size should be odd so that the label is in the middle
shape = (10, size, size) #define shape of features
labels1 = np.ones(1) #array for labels
data1 = np.ones(shape) #array for features
data1 = np.expand_dims(data1, axis=0) #expand dimension to concatenate
for j in range(20): #iterate over images in directory
  if j < 10:
    X = np.load('/content/images/image_00'+ str(j) + '.npy')
    y = np.load('/content/masks/mask_00'+ str(j) + '.npy')
    indices = np.argwhere(y > 0) #select all values with label
    indices_2d = indices[:, 1:] #extract indices
    ind_y = np.ones(2).reshape(-1,2) #array to collect indices
    for i in indices_2d: #iterate over indices
      if shape == X[:, i[0] - (size//2):i[0] + (size//2) + 1, i[1] - (size//2):i[1] + (size//2) + 1].shape: #select only features with the same shape because of labels at the image border
        temp = X[:, i[0] - (size//2):i[0] + (size//2) + 1, i[1] - (size//2):i[1] + (size//2) + 1] #save them temporary
        temp2 = np.expand_dims(temp, axis=0) #expand dimension to concatenate
        data1 = np.concatenate((data1, temp2), axis=0) #concatenation

        ind_y = np.concatenate((ind_y, i.reshape(-1,2)), axis=0) #concatenation of index so that they have the same order and length as the features

    ind_y = ind_y[1:] #remove first dummy values
    indices_1 = ind_y[:, 0].astype(int)
    indices_2 = ind_y[:, 1].astype(int)
    data_y = y[0, indices_1, indices_2] #extract labels
    labels1 = np.concatenate((labels1, data_y), axis = 0) #concatenate labels

  if j >= 10:
    X = np.load('/content/images/image_0'+ str(j) + '.npy')
    y = np.load('/content/masks/mask_0'+ str(j) + '.npy')
    indices = np.argwhere(y > 0)
    indices_2d = indices[:, 1:]
    ind_y = np.ones(2).reshape(-1,2)
    for i in indices_2d:
      if shape == X[:, i[0] - (size//2):i[0] + (size//2) + 1, i[1] - (size//2):i[1] + (size//2) + 1].shape:
        temp = X[:, i[0] - (size//2):i[0] + (size//2) + 1, i[1] - (size//2):i[1] + (size//2) + 1]
        temp2 = np.expand_dims(temp, axis=0)
        data1 = np.concatenate((data1, temp2), axis=0)

        ind_y = np.concatenate((ind_y, i.reshape(-1,2)), axis=0)

    ind_y = ind_y[1:]
    indices_1 = ind_y[:, 0].astype(int)
    indices_2 = ind_y[:, 1].astype(int)
    data_y = y[0, indices_1, indices_2]
    labels1 = np.concatenate((labels1, data_y), axis = 0)

data1 = data1[1:] #remove first dummy values
labels1 = labels1[1:] #remove first dummy values

In [ ]:
!unzip gdrive/My\ Drive/DataDa22/images_02.zip #second part of the data

In [ ]:
!unzip gdrive/My\ Drive/DataDa22/masks_02.zip

In [9]:
size = 5
shape = (10, size, size)
labels2 = np.ones(1)
data2 = np.ones(shape)
data2 = np.expand_dims(data2, axis=0)
for j in range(20):
  if j < 10:
    X = np.load('/content/images/image_00'+ str(j) + '.npy')
    y = np.load('/content/masks/mask_00'+ str(j) + '.npy')
    indices = np.argwhere(y > 0)
    indices_2d = indices[:, 1:]
    ind_y = np.ones(2).reshape(-1,2)
    for i in indices_2d:
      if shape == X[:, i[0] - (size//2):i[0] + (size//2) + 1, i[1] - (size//2):i[1] + (size//2) + 1].shape:
        temp = X[:, i[0] - (size//2):i[0] + (size//2) + 1, i[1] - (size//2):i[1] + (size//2) + 1]
        temp2 = np.expand_dims(temp, axis=0)
        data2 = np.concatenate((data2, temp2), axis=0)

        ind_y = np.concatenate((ind_y, i.reshape(-1,2)), axis=0)

    ind_y = ind_y[1:]
    indices_1 = ind_y[:, 0].astype(int)
    indices_2 = ind_y[:, 1].astype(int)
    data_y = y[0, indices_1, indices_2]
    labels2 = np.concatenate((labels2, data_y), axis = 0)

  if j >= 10:
    X = np.load('/content/images/image_0'+ str(j) + '.npy')
    y = np.load('/content/masks/mask_0'+ str(j) + '.npy')
    indices = np.argwhere(y > 0)
    indices_2d = indices[:, 1:]
    ind_y = np.ones(2).reshape(-1,2)
    for i in indices_2d:
      if shape == X[:, i[0] - (size//2):i[0] + (size//2) + 1, i[1] - (size//2):i[1] + (size//2) + 1].shape:
        temp = X[:, i[0] - (size//2):i[0] + (size//2) + 1, i[1] - (size//2):i[1] + (size//2) + 1]
        temp2 = np.expand_dims(temp, axis=0)
        data2 = np.concatenate((data2, temp2), axis=0)

        ind_y = np.concatenate((ind_y, i.reshape(-1,2)), axis=0)

    ind_y = ind_y[1:]
    indices_1 = ind_y[:, 0].astype(int)
    indices_2 = ind_y[:, 1].astype(int)
    data_y = y[0, indices_1, indices_2]
    labels2 = np.concatenate((labels2, data_y), axis = 0)

data2 = data2[1:]
labels2 = labels2[1:]

In [10]:
features = np.concatenate((data1, data2), axis = 0) #concatenate both parts
labels = np.concatenate((labels1, labels2), axis = 0)

In [13]:
print(features.shape)
print(labels.shape)

(38531, 10, 5, 5)
(38531,)


In [14]:
Xtrain, Xtest, ytrain, ytest = train_test_split(features, labels , test_size = 0.3, random_state=3) #create train, test set

In [58]:
%load_ext tensorboard

In [ ]:
modelCnn = Sequential() #bulid cnn
modelCnn.add(InputLayer(input_shape = (10, 5, 5)))
modelCnn.add(Conv2D(filters=50, kernel_size= (3,3), strides=  1 , padding = "valid", activation='relu' )) #padding valid
modelCnn.add(Conv2D(filters=100, kernel_size= (3,3), strides=  1 , padding = "valid", activation='relu' ))
modelCnn.add(Flatten())
modelCnn.add(Dense(100, activation='relu'))
modelCnn.add(Dense(50, activation='relu'))
modelCnn.add(Dense(1, activation='linear'))

modelCnn.summary()




In [ ]:
modelCnn.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_percentage_error']) #compile cnn
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(logdir, histogram_freq=1)

model_save = ModelCheckpoint("/content/gdrive/MyDrive/NNmodels/best_model.hdf5", save_best_only = True) #save best model

modelCnn.fit(Xtrain, ytrain, epochs = 50, validation_data=(Xtest, ytest), callbacks=[tensorboard_callback, model_save], batch_size = 128) #train cnn

In [70]:
bmodel = load_model('/content/gdrive/MyDrive/NNmodels/best_model.hdf5')
ypred_nn = bmodel.predict(Xtest) #predict best model


mse_nn = mse(ytest, ypred_nn) #calculate metrics
rmse_nn = mse_nn ** (1/2)
mae_nn = mae(ytest, ypred_nn)
mape_nn = mape(ytest, ypred_nn)

print(mape_nn)
print(mae_nn)
print(rmse_nn)

362/362 [==============================] - 2s 4ms/step
0.47492916277202396
4.265573303880989
7.078569066210429


In [ ]:
%tensorboard --logdir logs

In [72]:
mean_nn = np.mean(ypred_nn[:]) #calculate mean
quantiles_nn = np.percentile(ypred_nn[:], [1, 25, 50, 75, 99]) #calculate quantiles 0.01, 0.25, 0.5, 0.75, 0.99
mean_labels = np.mean(labels[:])
quantiles_labels = np.percentile(labels[:], [1, 25, 50, 75, 99])

print(mean_nn)
print(quantiles_nn)
print(np.sort(ypred_nn.flatten())[:10]) #print the 10 lowest predictions
print(np.sort(ypred_nn.flatten())[-10:][::-1]) #print the 10 highest predictions

print(mean_labels)
print(quantiles_labels)
print(np.sort(labels.flatten())[:10])
print(np.sort(labels.flatten())[-10:][::-1])

10.973388
[ 2.72695328  3.00690836  4.7546308  20.26180363 28.90485104]
[1.3616598 1.687053  1.8989965 2.0729659 2.129307  2.2452602 2.3526623
 2.3584313 2.3638072 2.417257 ]
[50.100033 39.6519   38.58132  34.773563 33.398247 32.901283 32.515965
 32.32575  31.560488 31.305954]
11.88373569284665
[ 2.3900001   2.8599999   5.6500001  20.79999924 39.44700012]
[1.34000003 1.57000005 1.75       1.83000004 1.89999998 1.89999998
 1.94000006 1.98000002 2.16000009 2.17000008]
[81.69000244 69.97000122 62.13000107 58.63000107 57.93000031 57.56000137
 55.20000076 54.56000137 54.11999893 54.04999924]
